In [45]:
#import local modules
from data_loading import get_or_create_database,load_dataset,load_paper_data,get_paper,get_papers
from preprocessor import ArXivPreprocessor
from vectorizer import Vectorizer,process_existing_papers,process_papers_batch
from rag import RAGAgent


#import third part modules
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import tool
from semantic_router.encoders import OpenAIEncoder
from pinecone import Pinecone,ServerlessSpec
from getpass import getpass


#import standard modules
import os
import time

In [14]:
# initialize connection to pinecone
api_key = os.getenv("PINECONE_API_KEY") or getpass("Pinecone API key: ")

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass("OpenAI API key: ")

#initialize encoder, pc, and spec. get dimension of the encoder as well
encoder = OpenAIEncoder(name="text-embedding-3-small")
pc = Pinecone(api_key=api_key)
spec = ServerlessSpec(
    cloud="aws", region="us-west-2"  # us-east-1
)

dims = len(encoder(["test"])[0])
print(dims)

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/home/bababako/side_projects/llm_projects/venv_rag/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


1536


In [15]:

#initialize the index. 
index_name = "dsny-rag-quick"

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=dims,  # dimensionality of embed 3
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 41}},
 'total_vector_count': 41}

# Data Ingestion

### Provide a dataset (e.g., JSON, CSV, or unstructured text files) that includes a mix of structured and unstructured data.

In [3]:
dataset = load_dataset("jamescalam/ai-arxiv2", split="train").shuffle(seed=42).select(range(20))

In [ ]:
dataset[2]

**We see here structured data, as well as unstructured data (the content, and to a limited degree, authors)**

### Ask the candidate to create a pipeline to load this data into a database of their choice, ensuring the schema is optimized for querying.

For the database, to not spend TOO much time on making the datbase, I handled Database Normilzation only up to the First Normal Form (1NF). For projects needing more complicated and in-depth setup, we can explore 2NF, 3NF, ... 5NF or other normilization technique. Also decided to use sqlite for the database for the simplicity of creation in the demo. **They are all found in models.py**

To optimize for querying, we handled the database with 1NF normalization, indexed frequent query columns, added constraints, and limited size of records.

Finally I decided to keep the full 'content' on the Paper in case there is a use case of needing the exact document returned.

In [ ]:
#example:
Session = get_or_create_database()
    
with Session() as session:
    paper_obj = get_papers(session=session,limit=1)[0]
    paper_dict = paper_obj.to_dict()

paper_dict


# 2. Data Preprocessing

### The data may contain noise or require transformation (e.g., text cleaning, parsing nested JSON, handling missing values).

### The candidate should demonstrate how they preprocess the data for efficient storage and later retrieval.

**The main player in data preprocessing is ArXivPreprocessor. It cleans up messy Unicode characters, removes unwanted special characters with regex, and converts raw academic papers into clean, standardized chunks. For storage efficiency, I've built in metadata tracking that includes token counts, document relationships (through pre/post chunk IDs), and citation references**

**There are two speeds, 0 and 1. 0 is a slower, more expensive approach but uses much more advanced algorithms to chunk the data (StatisticalChunking) otherwise we use a simple RollingWindowSplitter**

In [16]:
arxiv = ArXivPreprocessor(encoder=encoder,speed=1)


/home/bababako/side_projects/llm_projects/disney/RAG_Example_DSNY/preprocessor.py:47: UserWarning: Splitters are being deprecated. They have moved to their own package. Please migrate to the `semantic-chunkers` package. More information can be found at:
https://github.com/aurelio-labs/semantic-chunkers
  self.splitter = RollingWindowSplitter(


In [20]:
# example:
with Session() as session:
    chunks_data = arxiv.preprocess_paper(paper_obj)


2024-11-05 18:58:34 INFO semantic_router.utils.logger Single document exceeds the maximum token limit of 500. Splitting to sentences before semantically splitting.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:preprocessor:Successfully preprocessed paper 2401.04088 into 41 chunks


Splitting Statistics:
  - Total Documents: 238
  - Total Splits: 41
  - Splits by Threshold: 34
  - Splits by Max Chunk Size: 6
  - Last Split: 1
  - Minimum Token Size of Split: 43
  - Maximum Token Size of Split: 482
  - Similarity Split Ratio: 0.83


In [ ]:
chunks_data

# 3. Vectorization:

##### 1. Using a pre-trained language model or embeddings model, ask the candidate to convert the unstructured text into embeddings.
##### 2. Store these embeddings in a vector storage solution of their choice, ensuring the pipeline can handle batch processing for larger datasets.

**I've built a Vectorizer class that takes any embedding model and converts our preprocessed chunks into vectors, storing them in Pinecone with all our metadata intact and linked. Everything's built to handle scale - it processes in configurable batches using a batch_generator, has error handling, and shows real-time progress bars. Plus, storing vectors with their full context makes retrieval super smart later!**

In [22]:
vectorizer = Vectorizer(encoder=encoder,index=index,batch_size=5)

In [23]:
# example
vectorizer.vectorize_and_store(metadata_list=chunks_data)

100%|██████████| 9/9 [00:09<00:00,  1.04s/it]
INFO:vectorizer:Vectorization completed:
- Processed chunks: 41
- Failed chunks: 0
- Processing time: 9.45 seconds
- Rate: 4.34 chunks/second


{'total_chunks': 41,
 'processed_chunks': 41,
 'failed_chunks': 0,
 'processing_time': 9.452132,
 'chunks_per_second': 4.337645729027059}

# 4. Query and Retrieve:

##### 1. Create a simple API or script that allows querying based on a given text prompt.The query should retrieve similar embeddings from the vector store and return the corresponding records from the database.

##### 2. Include a use case for Retriever-Augmented Generation (RAG), where the retrieved data is used to generate a summary or response based on the query.

**There was a very simple way of avoiding the use of agents and custom vectorization by using langchain retrievers and vectorstores. Taking this approach would've let to a faster rag conversation. But I wanted to demonstrate my ability of making custom agents, and leaning towards my understanding of custom tools and control of the RAG environment. Albeit, this is still a extremely simple example of agent creation, and if needed for more complex demonstrations reach out to me through eneadodi.com** 

**Finally, because we have a database with very important information related to summary, citations, authors, etc we can use agents for things like:**
* finding documents of authors given a document used in response
* finding documents that cite the current document used in response
* finding and querying by category for comparison

**These can all be done by simply adding more tools on the tools list below.**


In [27]:

@tool 
def get_rag_context(query:str)->list[dict]:
    """
    Helper Tool Function that should always be called for an input by the user to get context.
    """
    return vectorizer.query(query)

tools = [get_rag_context]

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are a helpful AI assistant with access to relevant context through the get_rag_context tool. 
        Always use this tool first to retrieve context before answering questions.
        After getting context, use it to provide accurate and informed responses.
        If the context doesn't contain relevant information, acknowledge that and provide a general response.
        The context is largely in relation to science publications, so you know how to summarize those as an expert.
        Get it right and you get $100 tip!"""
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [37]:
agent = RAGAgent(vectorizer=vectorizer,tools=tools,prompt=prompt,verbose=True)
output = agent.query("what algorithms were used to find the results of the risk?")



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `get_rag_context` with `{'query': 'algorithms used to find the results of the risk'}`




INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[{'title': 'An Actor-Critic Algorithm for Sequence Prediction', 'content': 'ly, AC LL and RF-C LL for the combinations of AC and RF-C with LL. Character Error Rate AC LL 17.24 17.81 17.31 18.4 35.89 38.12 37.0 40.87 L 10, Î 0.3 L 30, Î 0.3 L 10, Î 0.5 L 30, Î 0.5 RF-C AC LL RF-C LL 17.82 18.16 35.84 37.6 16.65 17.1 34.6 36.36 16.97 17.47 35 36.6 Table 2: Our IWSLT 2014 machine translation results with a convolutional encoder compared to the previous work by Ranzato et al. Please see 1 for an explanation of abbreviations. The asterisk identiï es results from (Ranzato et al., 2015). The numbers reported with â were approximately read from Figure 6 of (Ranzato et al., 2015) Decoding method greedy search beam search LL MIXER 17.74 â 20.3 20.73 â 21.9 RF 20.92 21.35 RF-C 22.24 22.58 AC 21.66 22.45 out of four settings. In the fourth case, actor-critic and REINFORCE-critic have similar performance. Adding the log-likelihood gradient with a cofï cient Î LL 0.1 helps both of the methods, but a

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The context retrieved does not provide specific information about algorithms used to find the results of risk in a particular study. However, it does mention various algorithms related to sequence prediction and machine translation, such as:

1. **Actor-Critic Algorithm**: This is a reinforcement learning algorithm used for sequence prediction, which combines the benefits of both actor and critic networks.

2. **REINFORCE Algorithm**: A policy gradient method used in reinforcement learning, known for its high variance.

3. **SARSA and OLPOMDP**: Standard value-based reinforcement learning algorithms applied to structured prediction.

4. **Imitation Learning Algorithms**: Such as SEARN and DAGGER, which rely on an expert policy to provide action sequences for the policy to imitate.

5. **Minimum (Bayes) Risk Training**: An approach that replaces the domain over which the task score expectation is defined with a small subset, typically an n-best list or a sample.

If you have a specific 

# **FULL EXAMPLE**

In [35]:
#to load DIRECTLY from the database.
stats= process_existing_papers(preprocessor=arxiv,vectorizer=vectorizer,db_url='sqlite:///arxiv_papers.db',stop_at=10)



INFO:vectorizer:Found 10 papers to process
  0%|          | 0/10 [00:00<?, ?it/s]INFO:vectorizer:Starting preprocessing of 5 papers...
2024-11-05 19:37:54 INFO semantic_router.utils.logger Single document exceeds the maximum token limit of 500. Splitting to sentences before semantically splitting.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:preprocessor:Successfully preprocessed paper 1607.07086 into 49 chunks


Splitting Statistics:
  - Total Documents: 438
  - Total Splits: 49
  - Splits by Threshold: 31
  - Splits by Max Chunk Size: 17
  - Last Split: 1
  - Minimum Token Size of Split: 102
  - Maximum Token Size of Split: 500
  - Similarity Split Ratio: 0.63


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
100%|██████████| 10/10 [00:10<00:00,  1.07s/it]
INFO:vectorizer:Vectorization completed:
- Processed chunks: 49
- Failed chunks: 0
- Processing time: 

Splitting Statistics:
  - Total Documents: 3314
  - Total Splits: 252
  - Splits by Threshold: 185
  - Splits by Max Chunk Size: 66
  - Last Split: 1
  - Minimum Token Size of Split: 100
  - Maximum Token Size of Split: 500
  - Similarity Split Ratio: 0.73


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embedding

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embedding

Splitting Statistics:
  - Total Documents: 388
  - Total Splits: 51
  - Splits by Threshold: 39
  - Splits by Max Chunk Size: 11
  - Last Split: 1
  - Minimum Token Size of Split: 101
  - Maximum Token Size of Split: 498
  - Similarity Split Ratio: 0.76


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
100%|██████████| 11/11 [00:08<00:00,  1.33it/s]
INFO:vectorizer:V

Splitting Statistics:
  - Total Documents: 556
  - Total Splits: 76
  - Splits by Threshold: 57
  - Splits by Max Chunk Size: 18
  - Last Split: 1
  - Minimum Token Size of Split: 10
  - Maximum Token Size of Split: 498
  - Similarity Split Ratio: 0.75


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embedding

In [39]:
print(stats)

{'total_papers': 10, 'processed_papers': 10, 'failed_papers': 0, 'processing_time': 186.601471, 'papers_per_second': 0.053590145599656074}


In [46]:
agent = RAGAgent(vectorizer=vectorizer,tools=tools,prompt=prompt,verbose=False)

In [47]:
while True:
    user_input = input("🔍 Ask me anything (or 'exit()' to quit): ")
    
    if user_input.lower() == 'exit()':
        print("👋 Catch you later!")
        agent.clear_chat_history
        break
        
    try:
        output = agent.query(user_input)
        print("----------------------------\n")
        print("🤖"+output)
        print("\n----------------------------")
    except Exception as e:
        print(f"❌ Oops! Something went wrong: {str(e)}")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


----------------------------

🤖The results from algorithms used for abnormality detection, particularly in the context of out-of-distribution (OOD) detection and sequence prediction, include:

1. **Out-of-Distribution Detection**: The Maximum Softmax Probability (MSP) method was used to classify whether a test point belongs to an in-distribution dataset (e.g., ImageNet) or an out-of-distribution dataset (e.g., Places365). The performance of this binary classification task was evaluated using metrics such as the Area Under the Receiver Operating Characteristic (AUROC) and the Area Under the Precision-Recall Curve (AUPRC). These metrics help assess the model's ability to distinguish between in-distribution and out-of-distribution data points.

2. **Sequence Prediction**: In the context of sequence prediction, experiments were conducted to recover strings of natural text from corrupted versions, a task referred to as spelling correction. This task demonstrated the model's ability to handl

# 5. Documentation:

##### a. The candidate should document their code, the thought process behind their design choices, and any trade-offs they considered (e.g., schema design, vector storage approach, etc.).


**Documentation, including explanations of design choices, and code descriptions are found on the jupyter notebook and the files.** 


 

# 6. Bonus:

#####    Implement monitoring or logging for the data pipeline to track the data flow and identify potential bottlenecks.
#####    Optimize the pipeline for scalability, such as handling larger files or parallel processing.


**logging is implemented in all key aspects of preprocessing, vectorization, and data flow. Pipeline is optimized for scalability by handling work through batches where we handle offsets in a way that allows parallelization. Of course more work needs to be done to really use the batch handling correctly, and turning it into async functions as well. But for this very simple demonstration, I decided to leave out that work. If needed, reach out to me on eneadodi.com to ask for more complicated examples** 
 